In [1]:
import requests, json, csv
from datetime import datetime

# Endpoint
url = "https://graphql.coincap.io/"

# Query
query = """
{
  assets(first: 10) {
    edges {
      node {
        id
        symbol
        name
        rank
        priceUsd
        changePercent24Hr
        volumeUsd24Hr
        marketCapUsd
        supply
        vwapUsd24Hr
        website
        explorer
        logo
      }
    }
  }
}
"""

def safe_float(x):
    try: return float(x)
    except (TypeError, ValueError): return None

def fetch_crypto_data():
    response = requests.post(url, json={"query": query})
    if response.status_code != 200:
        raise Exception(f"API call failed: {response.status_code}\n{response.text}")
    data = response.json()
    assets = data["data"]["assets"]["edges"]
    cleaned = []
    for i in assets:
        a = i["node"]
        cleaned.append({
            "timestamp": datetime.utcnow().isoformat(),
            "id": a.get("id"),
            "symbol": a.get("symbol"),
            "name": a.get("name"),
            "rank": a.get("rank"),
            "priceUsd": safe_float(a.get("priceUsd")),
            "changePercent24Hr": safe_float(a.get("changePercent24Hr")),
            "volumeUsd24Hr": safe_float(a.get("volumeUsd24Hr")),
            "marketCapUsd": safe_float(a.get("marketCapUsd")),
            "supply": safe_float(a.get("supply")),
            "vwapUsd24Hr": safe_float(a.get("vwapUsd24Hr")),
            "website": a.get("website"),
            "explorer": a.get("explorer"),
            "logo": a.get("logo")
        })
    return cleaned

def save_to_json(data):
    file = f"coincap_data_{datetime.utcnow().strftime('%Y%m%d_%H%M%S')}.json"
    with open(file, "w") as f:
        json.dump(data, f, indent=4)
    print(f"✅ JSON saved: {file}")

def save_to_csv(data):
    file = f"coincap_data_{datetime.utcnow().strftime('%Y%m%d_%H%M%S')}.csv"
    keys = data[0].keys()
    with open(file, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=keys)
        writer.writeheader()
        writer.writerows(data)
    print(f"✅ CSV saved: {file}")

# Run once
crypto_data = fetch_crypto_data()
save_to_json(crypto_data)
save_to_csv(crypto_data)
print(f"✅ Collected and saved {len(crypto_data)} records successfully.")


✅ JSON saved: coincap_data_20251109_005206.json
✅ CSV saved: coincap_data_20251109_005206.csv
✅ Collected and saved 10 records successfully.
